In [1]:
from google.colab import files
files.upload() # kaggle.jsonをアップロード
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
import pandas as pd 
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
# 正規化用のクラスをインポート
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report as clf_report
from sklearn.impute import SimpleImputer

# データの準備
def prepare():
    !kaggle datasets download -d \
    dileep070/heart-disease-prediction-using-logistic-regression
    !unzip heart-disease-prediction-using-logistic-regression.zip
    df = pd.read_csv('framingham.csv')
    # 10年後の冠状動脈性心臓病CHDの発症リスクの有無
    # (0:無, 1:有)
    features = []
    # 分類に使用する特徴量（最後の列以外すべて）
    for i, f in enumerate(df.columns.values ):
        if i != len(df.columns)-1:
            features.append(f)
    X_train = df.loc[:,features].values
    y_train = df['TenYearCHD'].values
    return X_train, y_train, features

# 前処理（平均値による欠損値の補完、正規化）
def preprocess(X_train): 
    # 平均値による単一代入法
    simple_imp = SimpleImputer(missing_values=np.nan, strategy='mean') 
    simple_imp.fit(X_train ) 
    X_train = simple_imp.transform(X_train)
    ms = MinMaxScaler() 
    ms.fit(X_train)
    X_train = ms.transform(X_train)
    return X_train

# RFEによる特徴選択と学習
def select_by_rfe(n_features,features, X_train,y_train,X_test,y_test): 
    svc = SVC(kernel='linear', gamma=1/2 ,  
          C=1.0,class_weight='balanced',random_state=0)
    rfec = RFE(svc, n_features_to_select= \
              n_features,step=10, verbose=1)
    rfec.fit(X_train, y_train) 
    preds = rfec.predict(X_test) 
    print("RFE + SVC", rfec.n_features_)
    print(clf_report(y_test, preds, digits=3))
    feature_ranks = list(zip(rfec.ranking_, features)) 
    sorted_feature_ranks = sorted(
                           feature_ranks, reverse=False)
    # 選択された特徴量を表示する
    for i, fs in enumerate(sorted_feature_ranks[:n_features]):
        print('[%d]: %s\t%.2lf' % (i+1, fs[1], fs[0]))

def main(): 
    # n_features個に絞り込む
    n_features = 8
    X_train, y_train, features = prepare()
    X_train = preprocess(X_train)
    X_train, X_test, y_train, y_test = train_test_split(
               X_train, y_train, random_state=0, train_size=0.8)
    # SVMによる学習・分類（特徴選択なし）
    print('- training SVM with default parameters -')
    svc = SVC(kernel='linear', gamma=1/2,
              C=1.0, class_weight='balanced', random_state=0) 
    svc.fit(X_train, y_train)
    pred = svc.predict(X_test)
    print(clf_report(y_test, pred))
    # SVM-RFEを用いて特徴選択し、
    # n-features個の特徴量で分類、学習
    print('-*-*-*- Select by SVM-RFE -*-*-*-')
    select_by_rfe(n_features, features, X_train, y_train,
                  X_test, y_test)

if __name__ == '__main__':
    main()


  0% 0.00/58.4k [00:00<?, ?B/s]
100% 58.4k/58.4k [00:00<00:00, 22.2MB/s]
Archive:  heart-disease-prediction-using-logistic-regression.zip
  inflating: framingham.csv          
- training SVM with default parameters -
              precision    recall  f1-score   support

           0       0.92      0.66      0.77       710
           1       0.28      0.70      0.40       138

    accuracy                           0.67       848
   macro avg       0.60      0.68      0.59       848
weighted avg       0.81      0.67      0.71       848

-*-*-*- Select by SVM-RFE -*-*-*-
Fitting estimator with 15 features.
RFE + SVC 8
              precision    recall  f1-score   support

           0      0.922     0.662     0.770       710
           1      0.290     0.710     0.412       138

    accuracy                          0.670       848
   macro avg      0.606     0.686     0.591       848
weighted avg      0.819     0.670     0.712       848

[1]: BMI	1.00
[2]: age	1.00
[3]: cigsPerDay	1.0